## Example1

PDE:  
$$\frac{\partial^2u}{\partial x^2}-\frac{\partial^4u}{\partial y^4}=(2-x^2)e^{-y}$$
Boundary Condition:  
$$u_{yy}(x,0)=x^2$$
$$u_{yy}(x,1)=\frac{x^2}{e}$$
$$u(x,0)=x^2$$
$$u(x,1)=\frac{x^2}{e}$$
$$u(0,y)=0$$
$$u(1,y)=e^{-y}$$

## 1. import python modules and setup random seeds

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def set_seeds(seeds):
    torch.manual_seed(seeds)
    torch.cuda.manual_seed(seeds)
    torch.backends.cudnn.deterministic = True

set_seeds(12)

## 2. build nn model and define loss

In [13]:
class MLP(nn.Module):
    def __init__(self) -> None:
        super(MLP, self).__init__()
        self.model = torch.nn.Sequential(
            nn.Linear(2, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)


## 3. loss function

In [18]:
# mean square loss
mse_loss = nn.MSELoss()


# gradient
def gradient(u, x, order=1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                                   create_graph=True, only_inputs=True)[0]
    else:
        return gradient(gradient(u, x), x, order-1)


def loss_interior(model, n, device):
    """
    Parameters:
        n: number of interior points
    """

    x = torch.rand(n, 1, requires_grad=True).to(device=device)
    y = torch.rand(n, 1, requires_grad=True).to(device=device)
    cond = (2 - x ** 2) * torch.exp(-y)
    uxy = model(torch.cat([x, y], dim=1))

    return mse_loss(gradient(uxy, x, 2) - gradient(uxy, y, 4), cond)


def loss_uyy_down(model, n, device):
    """
    u_{yy}(x,0)=x^2
    """

    x = torch.randn(n, 1, requires_grad=True).to(device=device)
    y = torch.zeros_like(x, requires_grad=True).to(device=device)
    cond = x ** 2
    uxy = model(torch.cat([x, y], dim=1))

    return mse_loss(gradient(uxy, y, 2), cond)


def loss_uyy_up(model, n, device):
    """
    u_{yy}(x,1)=x^2 / e
    """
    
    x = torch.randn(n, 1, requires_grad=True).to(device=device)
    y = torch.ones_like(x, requires_grad=True).to(device=device)
    cond = x ** 2 / torch.e
    uxy = model(torch.cat([x, y], dim=1))
    
    return mse_loss(gradient(uxy, y, 2), cond)


def loss_u_down(model, n, device):
    """
    u(x,0)=x^2
    """

    x = torch.randn(n, 1, requires_grad=True).to(device=device)
    y = torch.zeros_like(x, requires_grad=True).to(device=device)
    cond = x ** 2
    uxy = model(torch.cat([x, y], dim=1))
    
    return mse_loss(uxy, cond)


def loss_u_up(model, n, device):
    """
    u(x,1)=x^2 / e
    """
    
    x = torch.randn(n, 1, requires_grad=True).to(device=device)
    y = torch.ones_like(x, requires_grad=True).to(device=device)
    cond = x ** 2 / torch.e
    uxy = model(torch.cat([x, y], dim=1))

    return mse_loss(uxy, cond)


def loss_u_left(model, n, device):
    """
    u(0,y)=0
    """
    
    y = torch.randn(n, 1, requires_grad=True).to(device=device)
    x = torch.zeros_like(y, requires_grad=True).to(device=device)
    cond = torch.zeros_like(x)
    uxy = model(torch.cat([x, y], dim=1))

    return mse_loss(uxy, cond)


def loss_u_right(model, n, device):
    """
    u(1,y)=e^{-y}
    """
    
    y = torch.randn(n, 1, requires_grad=True).to(device=device)
    x = torch.ones_like(y, requires_grad=True).to(device=device)
    cond = torch.exp(-y)
    uxy = model(torch.cat([x, y], dim=1))

    return mse_loss(uxy, cond)


def loss_data(model, n, device):
    x = torch.rand(n, 1, requires_grad=True).to(device=device)
    y = torch.rand(n, 1, requires_grad=True).to(device=device)
    cond = (x ** 2) * torch.exp(-y)
    uxy = model(torch.cat([x, y], dim=1))

    return mse_loss(uxy, cond)

## 4. training

In [20]:
# training parameters
num_epochs = 10000
device = "cuda:0" if torch.cuda.is_available() else "cpu"

n1 = 100 # 画图网格密度
n2 = 1000 # 内点配点数
n3 = 1000 # 边界点配点数
n4 = 1000 # PDE配点数

model = MLP().to(device=device)

optimizer = torch.optim.Adam(params=model.parameters())

for epoch in range(num_epochs):
    optimizer.zero_grad()

    loss =  loss_interior(model, n2, device) + loss_uyy_down(model, n3, device) + \
            loss_uyy_up(model, n3, device) + loss_u_down(model, n3, device) + \
            loss_u_up(model, n3, device) + loss_u_left(model, n3, device) + \
            loss_u_right(model, n3, device) # + loss_data(model, n4, device)
    loss.backward()
    
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch: ", epoch, " loss: ", loss.item())


Epoch:  0  loss:  14.266804695129395
Epoch:  100  loss:  6.713096618652344
Epoch:  200  loss:  1.446716547012329
Epoch:  300  loss:  3.2992024421691895
Epoch:  400  loss:  0.6489135026931763
Epoch:  500  loss:  0.14413265883922577
Epoch:  600  loss:  0.8584761619567871
Epoch:  700  loss:  1.0622957944869995
Epoch:  800  loss:  0.6996666193008423
Epoch:  900  loss:  0.5598262548446655
Epoch:  1000  loss:  0.7370679974555969
Epoch:  1100  loss:  0.5634816288948059
Epoch:  1200  loss:  0.4598393738269806
Epoch:  1300  loss:  0.022159745916724205
Epoch:  1400  loss:  0.2496308535337448
Epoch:  1500  loss:  0.04230286180973053
Epoch:  1600  loss:  0.0632500946521759
Epoch:  1700  loss:  0.02972234971821308
Epoch:  1800  loss:  0.021570825949311256
Epoch:  1900  loss:  0.19741007685661316
Epoch:  2000  loss:  0.09879502654075623
Epoch:  2100  loss:  0.025993939489126205
Epoch:  2200  loss:  0.07610786706209183
Epoch:  2300  loss:  0.005917555186897516
Epoch:  2400  loss:  0.00669271266087889

## 5. inference and visualization

In [21]:
xc = torch.linspace(0, 1, 100)
xm, ym = torch.meshgrid(xc, xc)
xx = xm.reshape(-1, 1)
yy = ym.reshape(-1, 1)
xy = torch.cat([xx, yy], dim=1)

u_pred = model(xy)
u_real = xx * xx * torch.exp(-yy)
u_error = torch.abs(u_pred - u_real)

u_pred_fig = u_pred.reshape(n1, n1)
u_real_fig = u_real.reshape(n1, n1)
u_error_fig = u_error.reshape(n1, n1)

print("Max abs error : ", float(torch.max(u_error)))

fig = plt.figure(1)
ax = Axes3D(fig)
ax.plot_surface(xm.detach().numpy(), ym.detach().numpy(), u_error_fig.detach().numpy())
ax.text2D(0.5, 0.9, "Abs Error", transform=ax.transAxes)
plt.show()
fig.savefig("Abs Error.png")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)

## References

[1] https://www.bilibili.com/video/BV1MP41187M3/?share_source=copy_web&vd_source=ac61c95531df6011c6660e9f78d47740